# MTH 371 Homework #1
Saeah Go \\
Due January 19, 2022 at 11:59 PM

# **Instruction** 
Given a unit lower triangular matrix L and an upper triangular matrix U. \\
(1) Write the forward and backward elimination solves ($Ly = b$ and $Ux = y$) assuming that L and U are stored in CSR format. Estimate the cost of memory and operations. \\
(2) Implement these algorithms using Python with the matrices L and U stored in CSR format. To get test matrices, proceed as follows: \\
> (2a) From the test matrices posted in Canvas, factor them into $LD^{-1}L^T$ form, either using some library, or implementing it yourself. \\
> (2b) Store the resulting unit triangular matrix L  in CSR format (do not store the zeros). \\

The posted matrices are stored in the adjacency list (also called coordinate) format: \\
i.e., in each row of the file we have three numbers: \\
$i ,j, a_{ij}$ \\
row index ($i$), column index ($j$) and value ($a_{ij}$) \\
The file contains only the nonzero entries of the matrix. The data should give s.p.d. matrices. \\
The assignment is given in detail in file: Hw1-notebook.ipynb

**ALGORITHM** *$ForwardElimination(L,\;b)$* \\
$\;\;\;\;for\;i←1\;to\;d\;do$ \\
$\;\;\;\;\;\;\;\;t←b_i$ \\
$\;\;\;\;\;\;\;\;for\;j←1\;to\;i+1\;do$ \\
$\;\;\;\;\;\;\;\;\;\;\;\;t←t-L_{ij}x_j$ \\
$\;\;\;\;\;\;\;\;end$ \\
$\;\;\;\;\;\;\;\;x_i←t/L_{ii}$ \\
$\;\;\;\;end$ \\
$\;\;\;\;return\;x$

**ALGORITHM** *$BackwardElimination(U,\;y)$* \\
$\;\;\;\;for\;i←n\;to\;1\;do$ \\
$\;\;\;\;\;\;\;\;t←y_i$ \\
$\;\;\;\;\;\;\;\;for\;j←i+1\;to\;n\;do$ \\
$\;\;\;\;\;\;\;\;\;\;\;\;t←t-U_{ij}x_j$ \\
$\;\;\;\;\;\;\;\;end$ \\
$\;\;\;\;\;\;\;\;x_i←t/U_{ii}$ \\
$\;\;\;\;end$ \\
$\;\;\;\;return\;x$


**Cost of memory & operations** \\
The operation cost of the forward elimination is: approximately $O(n^2)$ since there are two loops in the function. \\
The memory cost of the forward elimination is: $O(n^2)$ as we use two matrices, which are our two input data $L$ and $b$. Output data is $n$ since we return the $x$ matrix. \\
The operation cost of the backward elimination is: We have total two loops for the backward elimination function. Thus the time complexity is $O(n^2)$ \\
The memory cost of the backward elimination is: $O(n^2)$, we have two matrices $U$ and $y$. Similar to the forward elimination, the output data is $n$. \\

In [ ]:
import numpy as np
import pandas # read the data
from scipy.linalg import lu, lu_factor, lu_solve
from scipy.sparse import coo_matrix, csc_matrix, random
from scipy.sparse.linalg import splu, spsolve, spsolve_triangular


# read the data file
def read_data(filepath):
    data = pandas.read_csv(filepath,
                         sep=" ",
                         header=None)
    data.columns = ["col", "row", "data"]    
    matrix = coo_matrix((data["data"], (data["col"], data["row"])))

    return matrix

# read the data file (coordinate list)
def read_data_coo(filepath):
    data = pandas.read_csv(filepath,
                         sep=" ",
                         header=None)
    data.columns = ["col", "row", "data"] 
    data.col = data.col - 1 # coo matrix starts from 1, not starts from 0. so update the column indices
    data.row = data.row - 1 # similar to the column indices, update the row indices as well
    matrix = coo_matrix((data["data"], (data["col"], data["row"])))

    return matrix

# LU factorization using existing package
def LU_factor(sparse_matrix):

  lu = splu(matrix.tocsc(),
          permc_spec="NATURAL",
          diag_pivot_thresh=0,
          options={"SymmetricMode":True})
  return lu.L , lu.U

### my code ###

def forward_subs(L, b):
  n = L.shape[0] # calculate the number of rows
  x = np.zeros(n) # create a all zero vector
  for i in range(n):
    temp = b[i]
    for j in range(i+1):
      temp -= L[i, j] * x[j]
    x[i] = temp/L[i, i]
  return x

def backward_sub(U, y):
  n = U.shape[0]
  x = np.zeros(n) # create a all zero vector
  for i in range(n-1, -1, -1):
    temp = y[i]
    for j in range(i+1, n):
      temp -= U[i,j] * x[j]
    x[i] = temp/U[i, i]
  return x

#######################


In [ ]:
# Test the solution with 25.txt
matrix = read_data('25.txt')

lower_25,upper_25 = LU_factor(matrix)

x_vector = np.ones((matrix.shape[0],1))  # ground truth is all 1 vector

b = matrix @ x_vector # Ax = b

y = forward_subs(lower_25,b) # Ly = b
x_solution = backward_sub(upper_25,y) # Ux = y
print("b is: ", b)
print("x_vector is: ", x_vector)
print("x_solution is: ", x_solution)
print('difference between output and ground truth is :',np.linalg.norm(x_vector - x_solution))

# store the resulting unit triangular matrix L in CSR format (do not store the zeros)
result_l = lower_25.tocsr()
print('\nthe resulting unit triangular matrix L is: \n', result_l)

b is:  [[-4.64460997e-04]
 [ 1.12895723e-01]
 [ 8.36421441e-01]
 [ 2.99923155e-01]
 [ 1.53810093e-02]
 [ 3.84215778e-01]
 [ 2.60353856e-01]
 [ 3.67704206e-01]
 [ 2.68253129e-01]
 [ 1.77827941e-01]
 [ 1.06527471e-01]
 [ 2.61364734e-01]
 [ 3.51876053e-02]
 [ 9.97306787e-02]
 [ 4.35694394e-02]
 [ 6.37795836e-02]
 [ 2.79383529e-01]
 [ 6.43751600e-02]
 [ 4.56263825e-02]
 [ 2.78554397e-02]
 [ 2.40104838e-02]
 [ 5.32475145e-02]
 [ 1.66153330e-02]
 [ 9.84075573e-01]
 [ 1.02290797e-02]]
x_vector is:  [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
x_solution is:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]
difference between output and ground truth is : 1.5553031891404222e-14

the resulting unit triangular matrix L is: 
   (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 0)	0.007816513373454582
  (3, 3)	1.0
  (4, 4)	1.0
  (5, 2)	0.0342081422250432
  (5, 5)	1.0
  (6,

In [ ]:
# Test the solution with 50.txt
matrix = read_data('50.txt')

lower_50,upper_50 = LU_factor(matrix)

x_vector = np.ones((matrix.shape[0],1))  # ground truth is all 1 vector

b = matrix @ x_vector

y = forward_subs(lower_50,b)
x_solution = backward_sub(upper_50,y)

print('difference between output and ground truth is :',np.linalg.norm(x_vector - x_solution))

# store the resulting unit triangular matrix L in CSR format (do not store the zeros)
result_l = lower_50.tocsr()
print('\nthe resulting unit triangular matrix L is: \n', result_l)

FileNotFoundError: ignored

In [ ]:
# Test the solution with 500.txt
matrix = read_data('500.txt')

lower_500,upper_500 = LU_factor(matrix)

x_vector = np.ones((matrix.shape[0],1))  # ground truth is all 1 vector

b = matrix @ x_vector

y = forward_subs(lower_500,b)
x_solution = backward_sub(upper_500,y)

print('difference between output and ground truth is :',np.linalg.norm(x_vector - x_solution))

# store the resulting unit triangular matrix L in CSR format (do not store the zeros)
result_l = lower_500.tocsr()
print('\nthe resulting unit triangular matrix L is: \n', result_l)

In [ ]:
# Test the solution with 5000.txt
matrix = read_data('5000.txt')

lower_5000,upper_5000 = LU_factor(matrix)

x_vector = np.ones((matrix.shape[0],1))  # ground truth is all 1 vector

b = matrix @ x_vector

y = forward_subs(lower_5000,b)
x_solution = backward_sub(upper_5000,y)

print('difference between output and ground truth is :',np.linalg.norm(x_vector - x_solution))

# store the resulting unit triangular matrix L in CSR format (do not store the zeros)
result_l = lower_5000.tocsr()
print('\nthe resulting unit triangular matrix L is: \n', result_l)

In [ ]:
# Test the solution with 16x16.coo.txt
matrix = read_data_coo('16x16.coo.txt')

lower_16,upper_16 = LU_factor(matrix)

x_vector = np.ones((matrix.shape[1],1))  # ground truth is all 1 vector

b = matrix @ x_vector

y = forward_subs(lower_16,b)
x_solution = backward_sub(upper_16,y)

print('difference between output and ground truth is :',np.linalg.norm(x_vector - x_solution))

# store the resulting unit triangular matrix L in CSR format (do not store the zeros)
result_l = lower_16.tocsr()
print('\nthe resulting unit triangular matrix L is: \n', result_l)

In [ ]:
# Test the solution with 64x64.coo.txt
matrix = read_data_coo('64x64.coo.txt')

lower_64,upper_64 = LU_factor(matrix)

x_vector = np.ones((matrix.shape[1],1))  # ground truth is all 1 vector

b = matrix @ x_vector

y = forward_subs(lower_64,b)
x_solution = backward_sub(upper_64,y)

print('difference between output and ground truth is :',np.linalg.norm(x_vector - x_solution))

# store the resulting unit triangular matrix L in CSR format (do not store the zeros)
result_l = lower_64.tocsr()
print('\nthe resulting unit triangular matrix L is: \n', result_l)

In [ ]:
# Test the solution with 1024x1024.coo.txt
matrix = read_data_coo('1024x1024.coo.txt')

lower_1024,upper_1024 = LU_factor(matrix)

x_vector = np.ones((matrix.shape[1],1))  # ground truth is all 1 vector

b = matrix @ x_vector

y = forward_subs(lower_1024,b)
x_solution = backward_sub(upper_1024,y)

print('difference between output and ground truth is :',np.linalg.norm(x_vector - x_solution))

# store the resulting unit triangular matrix L in CSR format (do not store the zeros)
result_l = lower_1024.tocsr()
print('\nthe resulting unit triangular matrix L is: \n', result_l)